**Name:** \_\_\_\_\_

**EID:** \_\_\_\_\_

# CS4487 - Tutorial 9
## Face Detection with CNNs

In this tutorial you will train a CNN to detect whether there is a face in a small image patch.

First we need to initialize Python.  Run the below cell.

In [1]:
%matplotlib inline
import IPython.core.display         
# setup output image format (Chrome works best)
IPython.core.display.set_matplotlib_formats("svg")
import matplotlib.pyplot as plt
import matplotlib
from numpy import *
from sklearn import *
import glob
import os
import IPython.utils.warn as warn
random.seed(100)
import skimage.io
import skimage.color
import skimage.transform
from scipy import ndimage

C:\Anaconda2\lib\site-packages\sklearn\lda.py:4: DeprecationWarning: lda.LDA has been moved to discriminant_analysis.LinearDiscriminantAnalysis in 0.17 and will be removed in 0.19
  "in 0.17 and will be removed in 0.19", DeprecationWarning)
C:\Anaconda2\lib\site-packages\sklearn\qda.py:4: DeprecationWarning: qda.QDA has been moved to discriminant_analysis.QuadraticDiscriminantAnalysis in 0.17 and will be removed in 0.19.
  "in 0.17 and will be removed in 0.19.", DeprecationWarning)


Next we will load scikit-neuralnetwork and set 32-bit CPU mode.

In [2]:
from sknn.platform import cpu32
from sknn import mlp
import logging
logging.basicConfig()
import struct

ImportError: cannot import name gof

## 1. Loading Data and Pre-processing
Next we need to load the images.  Download `faces.zip`, and unzip it in the same directory as this ipynb file.  Then run the following cell to load the images.

In [ ]:
filelist = glob.glob('faces/*/*/*.png')

if len(filelist) == 0:
    warn.error("Could not find images in faces directory!  " + 
               "Make sure you put it here: " + os.getcwd() )
else:
    imgdata = {'train':[], 'test':[]}
    classes = {'train':[], 'test':[]}

    for f in filelist:
        # read image: range is [0,1]
        img = skimage.io.imread(f)
        # convert to grayscale
        img = skimage.color.rgb2gray(img)

        # filename is : faces/train/face/fname.png
        (fdir1, fname)  = os.path.split(f)     # get file name
        (fdir2, fclass) = os.path.split(fdir1) # get class (face, nonface)
        (fdir3, fset)   = os.path.split(fdir2) # get training/test set
        
        # class 1 = face; class 0 = non-face
        myclass = int(fclass == "face")  

        imgdata[fset].append(img)
        classes[fset].append(myclass)
    imgsize = img.shape

    
# remove some non-face test cases to balance the test set
testclass2start = sum(classes['test'])
imgdata['test']  = imgdata['test'][:2*testclass2start]
classes['test']  = classes['test'][:2*testclass2start]

Next we will convert the list of images into a block (array) of images for easier processing.

In [ ]:
# convert list to numpy array
trainY = asarray(classes['train'])  
testY  = asarray(classes['test'])

# convert list of ndarray to ndarray
trainI = asarray(imgdata['train'])
testI  = asarray(imgdata['test'])

# cleanup memory
del imgdata

# shuffle the data (since it is in order by class)
random.seed(123)
inds1 = random.permutation(len(trainI)).tolist()
inds2 = random.permutation(len(testI)).tolist()
trainY = trainY[inds1]
testY  = testY[inds2]
trainI = trainI[inds1]
testI = testI[inds2]

print trainI.shape
print testI.shape

Each image is a 19x19 array of pixel values.  Run the below code to show an example:

In [ ]:
print img.shape
plt.subplot(1,2,1)
plt.imshow(trainI[1], cmap='gray', interpolation='nearest')
plt.title("face sample")
plt.subplot(1,2,2)
plt.imshow(trainI[2], cmap='gray', interpolation='nearest')
plt.title("non-face sample")
plt.show()

Run the below code to show more images!

In [ ]:
# function to make an image montage
def image_montage(X, imsize=None, maxw=10):
    """X can be a list of images, or a matrix of vectorized images.
      Specify imsize when X is a matrix."""
    tmp = []
    numimgs = len(X)
    
    # create a list of images (reshape if necessary)
    for i in range(0,numimgs):
        if imsize != None:
            tmp.append(X[i].reshape(imsize))
        else:
            tmp.append(X[i])
    
    # add blanks
    if (numimgs > maxw) and (mod(numimgs, maxw) > 0):
        leftover = maxw - mod(numimgs, maxw)
        meanimg = 0.5*(X[0].max()+X[0].min())
        for i in range(0,leftover):
            tmp.append(ones(tmp[0].shape)*meanimg)
    
    # make the montage
    tmp2 = []
    for i in range(0,len(tmp),maxw):
        tmp2.append( hstack(tmp[i:i+maxw]) )
    montimg = vstack(tmp2) 
    return montimg

# show images in a plot
def show_imgs(W_list, nc=10, highlight_green=None, highlight_red=None, titles=None):
    # nc is the number of columns
    nfilter = len(W_list)
    nr = (nfilter - 1) // nc + 1
    for i in range(nr):
        for j in range(nc):
            idx = i * nc + j
            if idx == nfilter:
                break
            plt.subplot(nr, nc, idx + 1)
            cur_W = W_list[idx]
            plt.imshow(cur_W,cmap='gray', interpolation='nearest')  
            if titles is not None:
                if isinstance(titles, basestring):
                    plt.title(titles % idx)
                else:
                    plt.title(titles[idx])
            
            if ((highlight_green is not None) and highlight_green[idx]) or \
               ((highlight_red is not None) and highlight_red[idx]): 
                ax = plt.gca()
                if highlight_green[idx]:
                    mycol = '#00FF00'
                else:
                    mycol = 'r'
                for S in ['bottom', 'top', 'right', 'left']:
                    ax.spines[S].set_color(mycol)
                    ax.spines[S].set_lw(2.0)
                ax.xaxis.set_ticks_position('none')               
                ax.yaxis.set_ticks_position('none')
                ax.set_xticks([])
                ax.set_yticks([])
            else:
                plt.gca().set_axis_off()

# show a few images
plt.figure(figsize=(9,4))
plt.imshow(image_montage(trainI[trainY==0][0:50]), cmap='gray', interpolation='nearest')
plt.show()

plt.figure(figsize=(9,4))
plt.imshow(image_montage(trainI[trainY==1][0:50]), cmap='gray', interpolation='nearest')
plt.show()

Next we will generate the training/validation set from the training data.

In [ ]:
# generate fixed validation set of 10% of the training set
vtrainI, validI, vtrainY, validY = \
  cross_validation.train_test_split(trainI, trainY, 
  train_size=0.9, test_size=0.1, random_state=4488)

# make validation data
# - sknn expects the validation data shape to be: (N, C, Y, X)
# - N=number of images, C=color channels, Y=height, X=width
# we reshape the gray-scale (N, Y, X) to (N, 1, Y, X)
validsetI = (validI.reshape((validI.shape[0], 1, validI.shape[1], validI.shape[2])), 
             validY)

print vtrainI.shape
print validI.shape
print validsetI[0].shape

Next we define some useful functions for recording the training/validation error after each iteration.

In [ ]:
nnstats = {}  # dictionary for storing stats

def reset_stats(X, **_):
    # reset the statistics
    global nnstats
    nnstats['valid_error'] = []
    nnstats['train_error'] = []

def store_stats(avg_valid_error, avg_train_error, **_):
    # store the statistics after each iteration
    nnstats['valid_error'].append(avg_valid_error)
    nnstats['train_error'].append(avg_train_error)
    # print an update after 10 iterations
    if mod(len(nnstats['valid_error']), 10) == 0:
        print "iter %d: train=%0.4g; valid=%0.4g" % \
          (len(nnstats['valid_error']), nnstats['train_error'][-1], nnstats['valid_error'][-1])
    
def plot_nnstats(nnstats):    
    # plot the statistics
    plt.plot(nnstats['train_error'], label="training loss (%g)" % nnstats['train_error'][-1])
    plt.plot(nnstats['valid_error'], label="validation loss (%g)" % nnstats['valid_error'][-1])
    plt.grid(True)
    plt.legend(loc="best", fontsize=9)

# setup the callbacks
callbacks = {'on_epoch_finish': store_stats, 
             'on_train_start': reset_stats
            }

## 2. Detection using NN

Train a CNN to classify an image patch as face or not face.  Use  `vtrainI` and `vtrainY` as the training set and `validsetI` as the validation set.  You can try different architectures, and adjust values of the learning rates, number of iterations, weight decay, and dropout rate to get a good result.  Use a large batch size (e.g., 50) to speed up the training time.  Remember to add the `callbacks` so that you can monitor the training process.

In [ ]:
### INSERT YOUR CODE HERE



_How does the CNN compare to the linear and non-linear classifiers that you tried in Tutorial 4?_
- **INSERT YOUR ANSWER HERE**

## 3. Data Augmentation

Augmenting the training data with permutations is a good way to prevent NN from overfitting, and improving their generalization.  We will use two functions to augment the data in a batch when a batch starts, and reset the data when the batch ends.  A new callback dictionary `callbacks_aug` is defined to perform the augmentation during the training process.

In [ ]:
# global variable to save batch data before augmentation
savedXb = array([])

def augment_batch(Xb, yb, **vars):
    # Xb and yb are the data for the current batch   
    
    # first, make a copy of the original data 
    global savedXb
    savedXb = Xb.copy()  
    
    # second, add permutations to the data (you will write this function later)
    Xb[:] = add_noise(Xb) 

def reset_batch(Xb, yb, **vars):
    # reset the batch data to the saved data
    global savedXb
    Xb[:] = savedXb

# setup the callback dictionary
callbacks_aug = {'on_epoch_finish': store_stats, 
                 'on_train_start': reset_stats,
                 'on_batch_start': augment_batch, # augment at start of batch
                 'on_batch_finish': reset_batch,  # reset the batch data
               }

Now we define a few functions for adding noise or permuting the data.  The following functions are provided: 1) add Gaussian pixel noise; 2) add corruption noise (setting some input pixels to 0); 3) scale and shift pixel values (changing contrast and brightness); 4) mirror flip horizontally; 5) and translate horizontally and vertically.

In [ ]:
def add_gauss_noise(X, sigma2=0.03):
    # add Gaussian noise with zero mean, and variance sigma2
    return X + random.normal(0, sigma2, X.shape)

def add_corrupt_noise(X, p=0.1):
    # apply pixel corruption (zero out value) with probability p
    return X * random.binomial(1, 1-p, X.shape)

def add_scale_shift(X, sigma2=0.1, alpha2=0.2):
    # randomly scale and shift the pixel values (same for each image)
    # Xnew = a X + b
    # a is sampled from a Gaussian with mean 1, and variance sigma2
    # b is sampled from a Gaussian with mean 0, and variance alpha2
    if X.ndim == 3:
        dshape = (X.shape[0],1,1)
    elif X.ndim == 4:
        dshape = (X.shape[0],1,1,1)
    else:
        dshape = (1,)
    a = random.normal(1,sigma2, dshape)
    b = random.normal(0,alpha2, dshape)
    return minimum(maximum( a*X + b, 0.0), 1.0)

def add_flip(X):  
    # randomly horizontally flip all images with 50% probability
    f = random.binomial(1, 0.5)
    if f==1:
        return X[...,::-1]
    else:
        return X

def add_translate(X, maxT=2):
    # randomly translate all images vertically and horizontally
    # by -maxT to maxT.
    Tv = random.binomial(maxT*2, 0.5) - maxT;
    Th = random.binomial(maxT*2, 0.5) - maxT;
    return roll(roll(X,Tv,axis=-2), Th, axis=-1)

The code shows an example of these permutations. Run it several times to see different random effects.

In [ ]:
img = trainI[4]
noisefuncs = [add_gauss_noise, add_corrupt_noise, add_scale_shift, add_flip, add_translate]
imgs = [img]
titles = ['original image', 'Gaussian noise', 'corruption noise', 'scale and shift', 'flip', 'translate']
for f in noisefuncs:
    imgs.append(f(img))
plt.figure(figsize=(8,6))
show_imgs(imgs, nc=3, titles=titles)

Here is another example showing permutations of more data.

In [ ]:
# test methods on image stacks
imgbatch = trainI[0:20].copy()

plt.figure()
plt.imshow(image_montage(imgbatch), cmap='gray', interpolation='nearest')
plt.title('original')

for i,f in enumerate(noisefuncs):
    plt.figure()
    plt.imshow(image_montage(f(imgbatch)), cmap='gray', interpolation='nearest')
    plt.title(titles[i+1])

Train your best CNN from the previous section using data augmentation. To do this, you need to define the `add_noise` function, and then use `callbacks_aug` as the callback function when creating the NN.

Below is an example of an add_noise function. Try different types of data augmentation with different parameters, and combinations of them. Hopefully you should be able to improve the accuracy!  

In [ ]:
# here is an example add_noise function. It should return the permuted X.
def add_noise(X):
    return add_corrupt_noise(X, p=0.1)

In [ ]:
### INSERT YOUR CODE HERE ###

# NOTE: since we are permuting the data "in-place", you should train with
# a copy of the data so that you don't mess up the original dataset.
# Use the below call to the "fit" function
cnn.fit(vtrainI.copy(), vtrainY)

_Which type of augmentation improves the accuracy the most?  Why?_
- **INSERT YOUR ANSWER HERE**

# Test image
Now lets try your face detector on a real image.  Download the "nasa-small.png" image and put it in the same directory as your ipynb file.  The below code will load the image, crop out image patches and then extract features. (this may take a few minutes)

In [ ]:
fname = "nasa-small.png"

In [ ]:
# load image
testimg3 = skimage.io.imread(fname)

# convert to grayscale
testimg = skimage.color.rgb2gray(testimg3)
print testimg.shape
plt.imshow(testimg, cmap='gray')

In [ ]:
# step size for the sliding window
step = 4

# extract window patches with step size of 4
patches = skimage.util.view_as_windows(testimg, (19,19), step=step)
psize = patches.shape
# collapse the first 2 dimensions
patches2 = patches.reshape((psize[0]*psize[1], psize[2], psize[3]))
print patches2.shape 

# histogram equalize patches (improves contrast)
#newI = empty(patches2.shape)
#for i in range(patches2.shape[0]):
#    newI[i,:,:] = skimage.exposure.equalize_hist(patches2[i,:,:])
newI = patches2

Now predict using your classifier.  The extracted features are in `newXf`.

In [ ]:
### YOUR CODE HERE



Now we we will view the results on the image.  Use the below code. `prednewY` is the vector of predictions.

In [ ]:
# reshape prediction to an image
imgY = prednewY.reshape(psize[0], psize[1])

# zoom back to image size
imgY2 = ndimage.interpolation.zoom(imgY, step, output=None, order=0)
# pad the top and left with half the window size
imgY2 = vstack((zeros((9, imgY2.shape[1])), imgY2))
imgY2 = hstack((zeros((imgY2.shape[0],9)), imgY2))
# pad right and bottom to same size as image
if (imgY2.shape[0] != testimg.shape[0]):
    imgY2 = vstack((imgY2, zeros((testimg.shape[0]-imgY2.shape[0], imgY2.shape[1]))))
if (imgY2.shape[1] != testimg.shape[1]):
    imgY2 = hstack((imgY2, zeros((imgY2.shape[0],testimg.shape[1]-imgY2.shape[1]))))
    
# show detections with image
detimg = dstack(((0.5*imgY2+0.5)*testimg, 0.5*testimg, 0.5*testimg))

# show it!
plt.figure(figsize=(9,9))
plt.subplot(2,1,1)
plt.imshow(imgY2, interpolation='nearest')
plt.title('detection map')
plt.subplot(2,1,2)
plt.imshow(detimg)
plt.title('image')
plt.axis('image')

_How did your face detector do?_
- **INSERT YOUR ANSWER HERE**

- You can try it on your own images.  The faces should all be around 19x19 pixels though.
- We only used 1/8 of the training data. Try using more data to train it!